In [ ]:
# Setup the codespace
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from termcolor import colored

print(colored('\nAll libraries imported succesfully.', 'green'))

In [ ]:
# import datasets
test_data = pd.read_csv('test_data.csv')
train_data_control = pd.read_csv('train_data_control.csv')
train_data_control = pd.get_dummies(train_data_control, columns = ['target'])
kNN_imputed_10 = pd.read_csv('kNN_imputed_10.csv')
kNN_imputed_40 = pd.read_csv('kNN_imputed_40.csv')
kNN_imputed_70 = pd.read_csv('kNN_imputed_70.csv')

datasets = {
'train_data_control' : train_data_control,
'kNN_imputed_10' : kNN_imputed_10,
'kNN_imputed_40' : kNN_imputed_40,
'kNN_imputed_70' : kNN_imputed_70,
}




I ONLY JUST STARTED WITH THIS. THE TEST TARGET HAS NOT BEEN ONE HOT ENCODED YET. PERHAPS UNENCODE THE OTHERS, TO HAVE A BETTER CLASSIFICATION.

FIGURE OUT IF THIS WILL MAKE A DIFFERENCE

In [ ]:
# Functions
def kNN_class(train_data, test_data):
    evaluation = {}
    features = [col for col in train_data.columns if col != 'target' and col != 'ID']
    print(features)
    x_train = train_data[features]
    y_train = train_data.target

    features = [col for col in test_data.columns if col != 'target' and col != 'ID']
    print(features)
    x_test = test_data[features]
    y_test = test_data.target

    # Split dataset into training set and test set
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.4, random_state=1)

    # Determine optimal k
    k_values = [i for i in range (1,10)]
    max_score = 0
    opt_k = 0

    for k in k_values:
        knn = KNeighborsClassifier(n_neighbors=k)
        score = cross_val_score(knn, x_train, y_train, cv=4)
        mean_score = np.mean(score)

        if mean_score > max_score:
            max_score = mean_score
            opt_k = k

    #Set and fit model
    knn = KNeighborsClassifier(n_neighbors=opt_k)
    knn.fit(x_train, y_train)
    y_pred = knn.predict(x_test)

    # class_report = classification_report(y_test, y_pred)
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred,average='weighted')
    recall = recall_score(y_test, y_pred,average='weighted')
    f1 = f1_score(y_test, y_pred,average='weighted')
    # rocauc = roc_auc_score(y_test, y_pred)
    evaluation = {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1
        # 'rocauc': rocauc
    }
    return evaluation

def DecisionTree_class(train_data, test_data):
    evaluation = {}
    features = [col for col in train_data.columns if col != 'target' and col != 'ID']
    print(features)
    x_train = train_data[features]
    y_train = train_data.target

    features = [col for col in test_data.columns if col != 'target' and col != 'ID']
    print(features)
    x_test = test_data[features]
    y_test = test_data.target

    # Split dataset into training set and test set
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.4, random_state=1)
    clf = DecisionTreeClassifier()
    clf = clf.fit(x_train,y_train)
    y_pred = clf.predict(x_test)

    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred,average='weighted')
    recall = recall_score(y_test, y_pred,average='weighted')
    f1 = f1_score(y_test, y_pred,average='weighted')
 
    evaluation = {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1
        # 'rocauc': rocauc
    }
    return evaluation



# Collect classification results
results_classification = []
for dataset, data_items in datasets.items():
    try:
        results_classification.append({'dataset': dataset, **kNN_class(data_items)})
    except Exception as e:
        error_message = str(e).split('\n')[0]  
        print(f"Error occurred in k-NN Classification for dataset {dataset}: {error_message}")

    try:
        results_classification.append({'dataset': dataset, **DecisionTree_class(data_items)})
    except Exception as e:
        error_message = str(e).split('\n')[0] 
        print(f"Error occurred in Decision Tree Classification for dataset {dataset}: {error_message}")

results_classification_df = pd.DataFrame(results_classification)
display(results_classification_df)
